Data Pipelines en DVC --- 0:00 min
===

* 0:00 min | Ultima modificación: Abril 4, 2022 | YouTube

**No funciona a gran escala, pero este es un buen ejemplo de pipelines**

In [1]:
#
# Limpieza del área de trabajo
#
!rm -rf /tmp/pipeline /tmp/dvcstore

Creación de una carpeta temporal para el demo
---

In [2]:
!mkdir /tmp/pipeline
!mkdir /tmp/pipeline/data
%cd /tmp/pipeline

/tmp/pipeline


Inicializacion de git
---

In [3]:
!git init
!git config user.email jdvelasq@unal.edu.co
!git config user.name jdvelasq

Initialized empty Git repository in /tmp/pipeline/.git/


Inicialización de DVC
--

In [4]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


Descarga del proyecto ejemplo de DVC
---

In [5]:
!wget https://code.dvc.org/get-started/code.zip
!unzip code.zip
!rm -f code.zip
!ls -1

--2022-04-23 03:44:30--  https://code.dvc.org/get-started/code.zip
Resolving code.dvc.org (code.dvc.org)... 172.67.164.76, 104.21.81.205, 2606:4700:3033::ac43:a44c, ...
Connecting to code.dvc.org (code.dvc.org)|172.67.164.76|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://s3-us-east-2.amazonaws.com/dvc-public/code/get-started/code.zip [following]
--2022-04-23 03:44:31--  https://s3-us-east-2.amazonaws.com/dvc-public/code/get-started/code.zip
Resolving s3-us-east-2.amazonaws.com (s3-us-east-2.amazonaws.com)... 52.219.99.65
Connecting to s3-us-east-2.amazonaws.com (s3-us-east-2.amazonaws.com)|52.219.99.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5405 (5.3K) [application/zip]
Saving to: ‘code.zip’

code.zip            100%[===================>]   5.28K  --.-KB/s    in 0s      

2022-04-23 03:44:31 (206 MB/s) - ‘code.zip’ saved [5405/5405]

Archive:  code.zip
  inflating: params.yaml             
  inflating: src/e

Instalación paquetes
---

In [6]:
# !pip3 install --quiet -r src/requirements.txt
!cat src/requirements.txt

pandas
pyaml
scikit-learn
scipy


Stage prepare
---

In [7]:
cmd = """
dvc stage add -n prepare \
              -p prepare.seed,prepare.split \
              -d src/prepare.py \
              -d data/data.xml \
              -o data/prepared \
               python3 src/prepare.py data/data.xml
"""
!{cmd}

Creating 'dvc.yaml'                                                   core>
Adding stage 'prepare' in 'dvc.yaml'

To track the changes with git, run:

    git add data/.gitignore dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


In [8]:
!cat dvc.yaml

stages:
  prepare:
    cmd: python3 src/prepare.py data/data.xml
    deps:
    - data/data.xml
    - src/prepare.py
    params:
    - prepare.seed
    - prepare.split
    outs:
    - data/prepared


src/prepare.py
---

In [10]:
!pygmentize src/prepare.py

import io
import os
import random
import re
import sys
import xml.etree.ElementTree

import yaml

params = yaml.safe_load(open("params.yaml"))["prepare"]

if len(sys.argv) != 2:
    sys.stderr.write("Arguments error. Usage:\n")
    sys.stderr.write("\tpython prepare.py data-file\n")
    sys.exit(1)

# Test data set split ratio
split = params["split"]
random.seed(params["seed"])

input = sys.argv[1]
output_train = os.path.join("data", "prepared", "train.tsv")
output_test = os.path.join("data", "prepared", "test.tsv")


def process_posts(fd_in, fd_out_train, fd_out_test, target_tag):
    num = 1
    for line in fd_in:
        try:
            fd_out = fd_out_train if random.random() > split else fd_out_test
            attr = xml.etree.ElementTree.fromstring(line).attrib

            pid = attr.get("Id", "")
            label = 1 if target_tag in attr.get("Tags", "") else 0
            title = re.sub(r"\s+", " ", attr.get("Title", "")).strip()
            body = re.sub(r"\s+", " ", attr.

Stage featurize
---

In [11]:
cmd = """
dvc stage add -n featurize \
              -p featurize.max_features,featurize.ngrams \
              -d src/featurization.py \
              -d data/prepared \
              -o data/features \
              python3 src/featurization.py data/prepared data/features
"""
!{cmd}

Adding stage 'featurize' in 'dvc.yaml'                                core>

To track the changes with git, run:

    git add dvc.yaml data/.gitignore

To enable auto staging, run:

	dvc config core.autostage true


In [12]:
!cat dvc.yaml

stages:
  prepare:
    cmd: python3 src/prepare.py data/data.xml
    deps:
    - data/data.xml
    - src/prepare.py
    params:
    - prepare.seed
    - prepare.split
    outs:
    - data/prepared
  featurize:
    cmd: python3 src/featurization.py data/prepared data/features
    deps:
    - data/prepared
    - src/featurization.py
    params:
    - featurize.max_features
    - featurize.ngrams
    outs:
    - data/features


src/featurization.py
---

In [13]:
!pygmentize src/featurization.py

import os
import pickle
import sys

import numpy as np
import pandas as pd
import scipy.sparse as sparse
import yaml
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

params = yaml.safe_load(open("params.yaml"))["featurize"]

np.set_printoptions(suppress=True)

if len(sys.argv) != 3 and len(sys.argv) != 5:
    sys.stderr.write("Arguments error. Usage:\n")
    sys.stderr.write("\tpython featurization.py data-dir-path features-dir-path\n")
    sys.exit(1)

train_input = os.path.join(sys.argv[1], "train.tsv")
test_input = os.path.join(sys.argv[1], "test.tsv")
train_output = os.path.join(sys.argv[2], "train.pkl")
test_output = os.path.join(sys.argv[2], "test.pkl")

max_features = params["max_features"]
ngrams = params["ngrams"]


def get_df(data):
    df = pd.read_csv(
        data,
        encoding="utf-8",
        header=None,
        delimiter="\t",
        names=["id", "label", "text"],
    )
    sys.stderr.write(f"The input data frame {data} size is {df.sh

Stage train
---

In [14]:
cmd = """
dvc stage add -n train \
              -p train.seed,train.n_est,train.min_split \
              -d src/train.py \
              -d data/features \
              -o model.pkl \
              python3 src/train.py data/features model.pkl
"""
!{cmd}

Adding stage 'train' in 'dvc.yaml'                                    core>

To track the changes with git, run:

    git add .gitignore dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


In [15]:
!cat dvc.yaml

stages:
  prepare:
    cmd: python3 src/prepare.py data/data.xml
    deps:
    - data/data.xml
    - src/prepare.py
    params:
    - prepare.seed
    - prepare.split
    outs:
    - data/prepared
  featurize:
    cmd: python3 src/featurization.py data/prepared data/features
    deps:
    - data/prepared
    - src/featurization.py
    params:
    - featurize.max_features
    - featurize.ngrams
    outs:
    - data/features
  train:
    cmd: python3 src/train.py data/features model.pkl
    deps:
    - data/features
    - src/train.py
    params:
    - train.min_split
    - train.n_est
    - train.seed
    outs:
    - model.pkl


src/train.py
---

In [16]:
!pygmentize src/train.py

import os
import pickle
import sys

import numpy as np
import yaml
from sklearn.ensemble import RandomForestClassifier

params = yaml.safe_load(open("params.yaml"))["train"]

if len(sys.argv) != 3:
    sys.stderr.write("Arguments error. Usage:\n")
    sys.stderr.write("\tpython train.py features model\n")
    sys.exit(1)

input = sys.argv[1]
output = sys.argv[2]
seed = params["seed"]
n_est = params["n_est"]
min_split = params["min_split"]

with open(os.path.join(input, "train.pkl"), "rb") as fd:
    matrix = pickle.load(fd)

labels = np.squeeze(matrix[:, 1].toarray())
x = matrix[:, 2:]

sys.stderr.write("Input matrix size {}\n".format(matrix.shape))
sys.stderr.write("X matrix size {}\n".format(x.shape))
sys.stderr.write("Y matrix size {}\n".format(labels.shape))

clf = RandomForestClassifier(
    n_estimators=n_est, min_samples_split=min_split, n_jobs=2, random_state=seed
)

clf.fit(x, labels)

with open(output, "wb") as fd:
    pickle.dump(clf, fd)


Data
---

In [17]:
#
# Descarga los datos desde el repositorio de ejemplo de dvc
#
repo = "https://github.com/iterative/dataset-registry"
src = "get-started/data.xml"
dst = "data/data.xml"

!dvc get {repo} {src} -o {dst}

Reproducción
---

In [18]:
!dvc repro --quiet

The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)


In [19]:
!cat dvc.lock

schema: '2.0'
stages:
  prepare:
    cmd: python3 src/prepare.py data/data.xml
    deps:
    - path: data/data.xml
      md5: a304afb96060aad90176268345e10355
      size: 37891850
    - path: src/prepare.py
      md5: 51549a1c87b182ebdd785704f56ffaf1
      size: 1581
    params:
      params.yaml:
        prepare.seed: 20170428
        prepare.split: 0.2
    outs:
    - path: data/prepared
      md5: 20b786b6e6f80e2b3fcf17827ad18597.dir
      size: 23861319
      nfiles: 2
  featurize:
    cmd: python3 src/featurization.py data/prepared data/features
    deps:
    - path: data/prepared
      md5: 20b786b6e6f80e2b3fcf17827ad18597.dir
      size: 23861319
      nfiles: 2
    - path: src/featurization.py
      md5: 61c592707fd1b33e27819c87cf93f80a
      size: 2391
    params:
      params.yaml:
        featurize.max_features: 500
        featurize.ngrams: 1
    outs:
    - path: data/features
      md5: b4d13e5525577c1e4cbd31eafd1623d0.dir
      size: 7083901
      nfiles: 2
  train:
    

Visualización
---

In [20]:
!dvc dag

 +---------+   
 | prepare |   
 +---------+   
      *        
      *        
      *        
+-----------+  
| featurize |  
+-----------+  
      *        
      *        
      *        
  +-------+    
  | train |    
  +-------+    
